In [4]:
import cv2
import numpy as np

def compute_intersection_points(lines):
    intersection_points = []
    
    for i in range(len(lines)):
        rho1, theta1 = lines[i][0]
        
        for j in range(i+1, len(lines)):
            rho2, theta2 = lines[j][0]
            
            # Compute intersection point
            A = np.array([[np.cos(theta1), np.sin(theta1)], [np.cos(theta2), np.sin(theta2)]])
            b = np.array([rho1, rho2])
            
            try:
                x, y = np.linalg.solve(A, b)
                intersection_points.append((x, y))
            except np.linalg.LinAlgError:
                continue
                
    return intersection_points

# Load the image
image = cv2.imread('house1.png')

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray, 50, 150, apertureSize=3)

# Apply Hough line detection
lines = cv2.HoughLines(edges, 1, np.pi / 180, threshold=100)

# Compute intersection points of parallel lines
intersection_points = compute_intersection_points(lines)

# Find the average of all intersection points using the least squares method
avg_intersection_point = np.mean(intersection_points, axis=0)

# Display the image with detected lines and intersection points
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

for point in intersection_points:
    cv2.circle(image, (int(point[0]), int(point[1])), 5, (0, 255, 0), -1)

cv2.circle(image, (int(avg_intersection_point[0]), int(avg_intersection_point[1])), 10, (255, 0, 0), -1)

# Display the image with detected lines and intersection points
cv2.imshow('Intersection Points', image)
cv2.waitKey(0)
cv2.destroyAllWindows()
